In [1]:
import os
import sys
import json

path2folder = '/data/'
path2text = '../annotated_w_names/'
path2audio = '../data/'
txtFiles = os.listdir(path2text)
audioFiles = os.listdir(path2audio)
# restrict to .wav files
audioFiles = [f for f in audioFiles if f[-4:] == '.wav']
#filter text data
tempSet = set()
for elem in audioFiles:
    tempSet.add(elem[:-4])
txtFiles = [f for f in txtFiles if f.split('.')[0] in tempSet and f.split('.')[1][0:6] == 'en']
#filter audio to match text
tempSet = set()
for elem in txtFiles:
    tempSet.add(elem.split('.')[0])
audioFiles = [f for f in audioFiles if f[:-4] in tempSet]
# sort
txtFiles.sort()
audioFiles.sort()
# check
for i in range(len(txtFiles)):
    assert txtFiles[i].split('.')[0] == audioFiles[i][:-4]
print('Number of files: ', len(txtFiles))

Number of files:  56


In [2]:
# clean text
# for elem in txtFiles:
#     with open(path2text + elem, 'r') as f:
#         lines = f.readlines()
#     for i in range(len(lines)):
#         if lines[i][-2] != '>':
#             lines[i] = lines[i][:-1] + ' '
#     bigstring = ''.join(lines)
#     f = open(path2text + elem, 'w')
#     f.truncate(0)
#     f.write(bigstring)
#     f.close()

In [3]:
# parse text data and map text to speaker

def parseLine(line):
    line = line[1:-1]
    line = line.split('>')
    speaker = line[0]
    text = line[1].split('<')[0]
    return speaker, text.strip()
data = {}
for elem in txtFiles:
    data[elem] = {}
    with open(path2text + elem, 'r') as f:
        lines = f.readlines()
        for line in lines:
            speaker, text = parseLine(line)
            data[elem][text] = speaker

In [5]:
# takes an audio clip, and clips it, names it, and saves it for use in the TTS fine-tuning
def clipAudio(path2audio, path2save, start, end, name):
    os.system('ffmpeg -i ' + path2audio + ' -ss ' + str(start) + ' -to ' + str(end) + ' -c copy ' + path2save + name + '.wav')

path2json = '../transcripts/'
epdict = json.load(open(path2json + txtFiles[0].split('.')[0] + '.en.json3'))
def get_sentence(event):
    txt = ''
    try:
        for elem in event['segs']:
            txt += elem['utf8']
    except:
        pass
    return txt
joeyiter = 1
joeytxt = open('data/joey/metadata.txt', 'w')
connoriter = 1
connortxt = open('data/connor/metadata.txt', 'w')
garntiter = 1
garnttxt = open('data/garnt/metadata.txt', 'w')
for elem in txtFiles:
    epdict = json.load(open(path2json + elem.split('.')[0] + '.en.json3'))
    for i in range(len(epdict['events'])):
        txt = get_sentence(epdict['events'][i])
        if len(txt) > 3:
            try:
                speaker = data[elem][txt]
                starttime = epdict['events'][i]['tStartMs']/1000
                endtime = epdict['events'][i]['dDurationMs']/1000 + starttime
            except:
                print('Error: ', epdict['events'][i])
                continue
            if speaker == 'joey':
                clipAudio(path2audio + elem.split('.')[0] + '.wav', 'data/joey/wavs/', starttime, endtime, f'audio{joeyiter}')
                joeytxt.write(f'audio{joeyiter}|{txt}\n')
                joeyiter += 1
            elif speaker == 'connor':
                clipAudio(path2audio + elem.split('.')[0] + '.wav', 'data/connor/wavs/', starttime, endtime, f'audio{connoriter}')
                connortxt.write(f'audio{connoriter}|{txt}\n')
                connoriter += 1
            elif speaker == 'garnt':
                clipAudio(path2audio + elem.split('.')[0] + '.wav', 'data/garnt/wavs/', starttime, endtime, f'audio{garntiter}')
                garnttxt.write(f'audio{garntiter}|{txt}\n')
                garntiter += 1
garnttxt.close()
connortxt.close()
joeytxt.close()

